# Example of a GFMAP full-extraction pipeline

Designing of GFMAP Job DataFrames and DataCube creators functions, as well as post job-actions.

Those dataframe should be containing all the necessary infromation to run a job and know where to save it.

### First step: splitting the job

Splitting the dataset of extraction in multiple job based on position is necessary to respect OpenEO limitations.

This script performs a split with the H3 hexagonal grid, yielding a list of sub-geodataframes.

A subtility here is that some polygons are not directly extracted (field with `extract=False`), but should be kept for post-job actions. This requirement is filled by removing sub-dataframes that do not contain any extractable polyons.

In [1]:
# Configuring the logging for the openeo_gfmap package
from openeo_gfmap.manager import _log
import logging

_log.setLevel(logging.DEBUG)

stream_handler = logging.StreamHandler()
_log.addHandler(stream_handler)

formatter = logging.Formatter('%(asctime)s|%(name)s|%(levelname)s:  %(message)s')
stream_handler.setFormatter(formatter)

# Exclude the other loggers from other libraries
class MyLoggerFilter(logging.Filter):
    def filter(self, record):
        return record.name == _log.name

stream_handler.addFilter(MyLoggerFilter())


In [2]:
from pathlib import Path
import geopandas as gpd
from openeo_gfmap.manager.job_splitters import split_job_hex

base_df_path = Path('/vitodata/worldcereal/tmp/kristof/GFMAP/2021_EUR_DEMO_POLY_110.gpkg')
base_df = gpd.read_file(base_df_path)
# Splits the job using GFMAP
split_jobs = split_job_hex(
    base_df, max_points=60, grid_resolution=4
)

print(f'{len(split_jobs)} jobs before filtering empty one (no extraction)')

# Remove the geometry where there are no points with the "extract" flag
split_jobs = [
    job for job in split_jobs if job.extract.any()
]
print(f'{len(split_jobs)} jobs after filtering empty one (no extraction)')

base_df.head()


/data/users/Private/couchard/openeo-gfmap/src/openeo_gfmap/manager/job_splitters.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["h3index"] = polygons.geometry.centroid.apply(


334 jobs before filtering empty one (no extraction)
236 jobs after filtering empty one (no extraction)


,sample_id,landcover_label,croptype_label,irrigation_label,confidence,extract,valid_date,ref_id,geometry
0,2021_LV_LPIS_POLY_110-12553170,11,1520,0,None,False,2021-06-01,2021_EUR_DEMO_POLY_110,"MULTIPOLYGON (((22.64240 57.42064, 22.64241 57..."
1,at2021lpis207694,11,1520,0,None,False,2021-06-01,2021_EUR_DEMO_POLY_110,"MULTIPOLYGON (((15.87727 48.72497, 15.87735 48..."
2,2021_LV_LPIS_POLY_110-12765889,11,1520,0,None,False,2021-06-01,2021_EUR_DEMO_POLY_110,"MULTIPOLYGON (((27.14266 56.73090, 27.14278 56..."
3,2021_LV_LPIS_POLY_110-12558690,11,1520,0,None,True,2021-06-01,2021_EUR_DEMO_POLY_110,"MULTIPOLYGON (((25.47958 57.88641, 25.47956 57..."
4,at2021lpis778655,11,1520,0,None,False,2021-06-01,2021_EUR_DEMO_POLY_110,"MULTIPOLYGON (((14.72080 48.40937, 14.72020 48..."


### Second step: creating a dataframe for the GFMAP Job Manager

Implementing a function that yields a `pandas.DataFrame` where each row correponds to a job.

The dataframe should contain the informations required by the GFMAP Job Manager, as well as additional information used by the datacube creation function and the post-job action function.

The output dataframe should be savable as a .csv file.

Note: the full information of a sub-geodataframe of polygons can be saved into a row of a `pandas.DataFrame` by storing it in a row as string implementing the `geojson.FeatureCollection` interface. To convert the `geopandas.GeoDataFrame` into a stirng, simply use the `.to_json()` function.

In [3]:
from openeo_gfmap import Backend
from typing import List
import pandas as pd

def create_job_dataframe_s2(backend: Backend, split_jobs: List[gpd.GeoDataFrame]) -> pd.DataFrame:
    """Create a dataframe from the split jobs, containg all the necessary information to run the job."""
    columns = ['backend_name', 'out_prefix', 'out_extension', 'start_date', 'end_date', 'geometry']
    rows = []
    for job in split_jobs:
        # Compute the average in the valid date and make a buffer of 1.5 year around
        median_time = pd.to_datetime(job.valid_date).mean()
        start_date = median_time - pd.Timedelta(days=275)  # A bit more than 9 months
        end_date = median_time + pd.Timedelta(days=275)  # A bit more than 9 months
        
        rows.append(
            pd.Series(
                dict(zip(columns, [backend.value, 'S2_10m', '.nc',  start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), job.to_json()]))
            )
        )

    return pd.DataFrame(rows)

job_df = create_job_dataframe_s2(Backend.CDSE, split_jobs)

job_df

,backend_name,out_prefix,out_extension,start_date,end_date,geometry
0,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
4,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
...,...,...,...,...,...,...
231,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
232,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
233,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
234,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


In [4]:
# Run a subset of the jobs to test the manager, the selected jobs have a fair amount of geometries to extract
job_df = job_df.iloc[[0, 2, 3, -6]].reset_index(drop=True)
job_df

,backend_name,out_prefix,out_extension,start_date,end_date,geometry
0,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


In [5]:
import geojson

def get_job_nb_polygons(row: pd.Series) -> int:
    """Get the number of polygons in the geometry."""
    return len(filter(lambda feat: feat.properties.get("extract"), geojson.loads(row.geometry)['features']))

job_df['nb_polygons'] = job_df.apply(get_job_nb_polygons, axis=1)
job_df

,backend_name,out_prefix,out_extension,start_date,end_date,geometry,nb_polygons
0,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i...",50
1,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i...",60
2,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i...",20
3,cdse,S2_10m,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i...",38


### Third step: implement the datacube creator function.

Implement a function to create, from the additional rows provided before, an `openeo.BatchJob` that will be used to run the job.

In this case we extract Sentinel-2 data, and we remove the polygons with `extract=False` (although we keep them in the row for the post-job action.)

In [6]:
import openeo

import requests
from tempfile import NamedTemporaryFile
import os
import pandas as pd
import geojson
from shapely.geometry import Point

from openeo_gfmap import TemporalContext, Backend, BackendContext, FetchType, SpatialContext
from openeo_gfmap.fetching import build_sentinel2_l2a_extractor

def upload_geoparquet_artifactory(gdf: gpd.GeoDataFrame, row_id: int) -> str:
    # Save the dataframe as geoparquet to upload it to artifactory
    temporary_file = NamedTemporaryFile()
    gdf.to_parquet(temporary_file.name)
    
    artifactory_username = os.getenv('ARTIFACTORY_USERNAME')
    artifactory_password = os.getenv('ARTIFACTORY_PASSWORD')

    headers = {
        "Content-Type": "application/octet-stream"
    }

    upload_url = f"https://artifactory.vgt.vito.be/artifactory/auxdata-public/gfmap-temp/openeogfmap_dataframe_{row_id}.parquet"

    with open(temporary_file.name, 'rb') as f:
        response = requests.put(upload_url, headers=headers, data=f, auth=(artifactory_username, artifactory_password))

    assert response.status_code == 201, f"Error uploading the dataframe to artifactory: {response.text}"

    return upload_url


def create_datacube_s2(row: pd.Series, connection: openeo.DataCube, provider=None, connection_provider=None) -> openeo.BatchJob:

    def buffer_geometry(geometry: geojson.FeatureCollection, buffer: int) -> gpd.GeoDataFrame:
        gdf = gpd.GeoDataFrame.from_features(geometry).set_crs(epsg=4326)
        utm = gdf.estimate_utm_crs()
        gdf = gdf.to_crs(utm)

        gdf['geometry'] = gdf.centroid.apply(
            # Clips the point to the closest 20m from the S2 grid
            lambda point: Point(round(point.x / 20.0) * 20.0, round(point.y / 20.0) * 20.0)
        ).buffer(distance=buffer, cap_style=3)

        return gdf

    def filter_extractonly_geometries(collection: geojson.FeatureCollection):
        # Filter out geometries that do not have the field extract=True
        features = [f for f in collection.features if f.properties.get('extract', False)]
        return geojson.FeatureCollection(features)

    start_date = row.start_date
    end_date = row.end_date
    temporal_context = TemporalContext(start_date, end_date)

    # Get the feature collection containing the geometry to the job
    geometry = geojson.loads(row.geometry)
    assert isinstance(geometry, geojson.FeatureCollection)

    # Filter the geometry to the rows with the extract only flag
    geometry = filter_extractonly_geometries(geometry)
    assert len(geometry.features) > 0, "No geometries with the extract flag found"

    # Performs a buffer of 64 px around the geometry
    geometry_df = buffer_geometry(geometry, 320)
    spatial_extent_url = upload_geoparquet_artifactory(geometry_df, row.name)

    # Backend name and fetching type
    backend = Backend(row.backend_name)
    backend_context = BackendContext(backend)

    fetch_type = FetchType.POLYGON
    bands_to_download = ['S2-B01', 'S2-B02', 'S2-B03', 'S2-B04', 'S2-B05', 'S2-B06', 'S2-B07', 'S2-B08', 'S2-B8A', 'S2-B09', 'S2-B11', 'S2-B12', 'S2-SCL']

    # Create the job to extract S2
    extraction_parameters = {
        "target_resolution": 10,
        "load_collection": {
            "eo:cloud_cover": lambda val: val <= 95.0,
        },
    }
    extractor = build_sentinel2_l2a_extractor(
        backend_context, bands=bands_to_download, fetch_type=fetch_type.POLYGON, **extraction_parameters 
    )

    cube = extractor.get_cube(connection, spatial_extent_url, temporal_context)

    # Compute the SCL dilation and add it to the cube
    scl_dilated_mask = cube.process(
        "to_scl_dilation_mask",
        data=cube,
        scl_band_name="S2-SCL",
        kernel1_size=17,  # 17px dilation on a 20m layer
        kernel2_size=77,   # 77px dilation on a 20m layer
        mask1_values=[2, 4, 5, 6, 7],
        mask2_values=[3, 8, 9, 10, 11],
        erosion_kernel_size=3
    ).rename_labels("bands", ["S2-SCL_DILATED_MASK"])

    cube = cube.merge_cubes(scl_dilated_mask)
    cube = cube.linear_scale_range(0, 65534, 0, 65534)

    # Get the h3index to use in the tile
    h3index = geometry.features[0].properties['h3index']
    valid_date = geometry.features[0].properties['valid_date']

    # Increase the memory of the jobs depending on the number of polygons to extract
    number_polygons = get_job_nb_polygons(row)
    _log.debug(f"Number of polygons to extract: {number_polygons}")

    job_options = {
        "executor-memory": "5G",
        "executor-memoryOverhead": "2G",
    }

    return cube.create_job(
        out_format="NetCDF",
        title=f"GFMAP_Extraction_S2_{h3index}_{valid_date}",
        sample_by_feature=True,
        job_options=job_options
    )


### Fourth step: create output paths

Implement a function that from a temporary path containing a job result, from the job dataframe row and the root folder will choose the output path where to save that job result.

In [7]:
%%time

# Load the S2 grid
s2_grid = gpd.read_file('./s2grid_bounds.geojson')

CPU times: user 4.51 s, sys: 48 ms, total: 4.56 s
Wall time: 4.56 s


In [8]:
from pathlib import Path
import xarray as xr
from pyproj import Transformer, CRS
from shapely.geometry import box, Point

def generate_output_path_s2(root_folder: Path, tmp_path: Path, geometry_index: int, row: pd.Series):
    features = geojson.loads(row.geometry)
    sample_id = features[geometry_index].properties['sample_id']
    ref_id = features[geometry_index].properties['ref_id']
    
    # Loads the array lazily in-memory
    try:
        inds = xr.open_dataset(tmp_path, chunks='auto')
        
        source_crs = CRS.from_wkt(inds.crs.attrs['crs_wkt'])
        dst_crs = CRS.from_epsg(4326)
        
        transformer = Transformer.from_crs(source_crs, dst_crs, always_xy=True)

        # Get the center point of the tile
        centroid_utm = box(
            inds.x.min().item(), inds.y.min().item(), inds.x.max().item(), inds.y.max().item()
        ).centroid
        centroid_latlon = Point(*transformer.transform(centroid_utm.x, centroid_utm.y))

        # Intersecting with the s2 grid
        intersecting = s2_grid.geometry.intersects(centroid_latlon)

        # Select the intersecting cell that has a centroid the closest from the point
        intersecting_cells = s2_grid[intersecting]
        intersecting_cells['distance'] = intersecting_cells.distance(centroid_latlon)
        intersecting_cells.sort_values('distance', inplace=True)
        s2_tile = intersecting_cells.iloc[0]

        s2_tile_id = s2_tile.tile

        subfolder = root_folder / ref_id / str(source_crs.to_epsg()) / s2_tile_id / sample_id
    except Exception:
        subfolder = root_folder / 'unsortable'

    return subfolder / f'{row.out_prefix}_{sample_id}_{source_crs.to_epsg()}_{row.start_date}_{row.end_date}{row.out_extension}'
    

### Fifth step: Define the post-job action

The post-job action will be called once the job resut was downloaded and saved to a specific path.

A post-job action function must receive 3 parameters:
* `result_paths`: Paths to the downloaded job result files.
* `row`: The current job dataframe row.
* `parameters`: User-defined parameters set in the `GFMAPJobManager` constructor.

The post-job action must return a list of paths containing the results from that job. For example, if no file is created/deleted in the post-job action, then the user can simply return the list of paths it has received as input `result_paths`. If instead files are added or removed, then the user will need to modify this list accordingly before returning it.

In [9]:
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import json

def post_job_action(result_paths: list, row: pd.Series, parameters: dict = {}) -> list:
    base_gpd = gpd.GeoDataFrame.from_features(json.loads(row.geometry)).set_crs(epsg=4326)
    assert len(base_gpd[base_gpd.extract == True]) == len(result_paths), "The number of result paths should be the same as the number of geometries"
    extracted_gpd = base_gpd[base_gpd.extract == True].reset_index(drop=True)
    # In this case we want to burn the metadata in a new file in the same folder as the S2 product
    for idx, result_path in enumerate(result_paths.copy()):
        sample_id = extracted_gpd.iloc[idx].sample_id
        ref_id = extracted_gpd.iloc[idx].ref_id
        confidence = extracted_gpd.iloc[idx].confidence
        valid_date = extracted_gpd.iloc[idx].valid_date

        result_ds = xr.open_dataset(result_path, chunks='auto')

        target_crs = CRS.from_wkt(result_ds.crs.attrs['crs_wkt'])

        # Get the surrounding polygons around our extracted center geometry to rastetize them
        bounds = (result_ds.x.min().item(), result_ds.y.min().item(), result_ds.x.max().item(), result_ds.y.max().item())
        bbox = box(*bounds)
        surround_gpd = base_gpd.to_crs(target_crs).clip(bbox)

        # Burn the polygon croptypes
        transform = from_bounds(*bounds, result_ds.x.size, result_ds.y.size)
        croptype_shapes = list(zip(surround_gpd.geometry, surround_gpd.croptype_label))

        fill_value = 0
        croptype = rasterize(croptype_shapes, out_shape=(result_ds.y.size, result_ds.x.size), transform=transform, all_touched=False, fill=fill_value, default_value=0, dtype='int64')

        # Create the attributes to add to the metadata
        attributes = {
            'ref_id': ref_id,
            'sample_id': sample_id,
            'confidence': str(confidence),
            'valid_date': valid_date,
            '_FillValue': fill_value,
            'Conventions': 'CF-1.9',
        }

        aux_dataset = xr.Dataset(
            {'LABEL': (('y', 'x'), croptype), 'crs': result_ds['crs']},
            coords={'y': result_ds.y, 'x': result_ds.x},
            attrs=attributes
        )
        # Required to map the 'crs' layer as geo-reference for the 'LABEL' layer.
        aux_dataset['LABEL'].attrs['grid_mapping'] = 'crs'

        # Save the metadata in the same folder as the S2 product
        metadata_path = result_path.parent / f'WORLDCEREAL_10m_{sample_id}_{target_crs.to_epsg()}_{valid_date}.nc'
        aux_dataset.to_netcdf(metadata_path, format='NETCDF4', engine='h5netcdf')
        result_paths.append(metadata_path)

    return result_paths


### Sixth and last step: Running the manager

Let's initialize and execute the Job Manager as defined the GFMAP, and then run it using the functions defined previously

In [10]:
from openeo_gfmap.manager.job_manager import GFMAPJobManager
from openeo_gfmap.backend import cdse_connection

base_output_dir = Path('/data/users/Public/couchard/world_cereal/extractions_4/')
tracking_job_csv = base_output_dir / 'job_tracker.csv'

manager = GFMAPJobManager(
    output_dir=base_output_dir,
    output_path_generator=generate_output_path_s2,
    post_job_action=post_job_action,
    poll_sleep=60,
    n_threads=2,
    post_job_params={}
)

manager.add_backend(
    Backend.CDSE.value, cdse_connection, parallel_jobs=6
)

In [11]:
manager.run_jobs(job_df, create_datacube_s2, tracking_job_csv)

2024-03-01 10:09:05,568|openeo_gfmap.manager|INFO:  Starting job manager using 2 worker threads.
2024-03-01 10:09:05,574|openeo_gfmap.manager|INFO:  Workers started, creating and running jobs.
2024-03-01 10:09:06,100|openeo_gfmap.manager|DEBUG:  Normalizing dataframe. Columns: Index(['backend_name', 'out_prefix', 'out_extension', 'start_date', 'end_date',
       'geometry', 'nb_polygons', 'status', 'id', 'start_time', 'cpu',
       'memory', 'duration', 'description', 'costs'],
      dtype='object')
2024-03-01 10:09:06,106|openeo_gfmap.manager|DEBUG:  Updating status. 0 on 4 active jobs... -> DF columns: Index(['backend_name', 'out_prefix', 'out_extension', 'start_date', 'end_date',
       'geometry', 'nb_polygons', 'status', 'id', 'start_time', 'cpu',
       'memory', 'duration', 'description', 'costs'],
      dtype='object')


Authenticated using refresh token.


2024-03-01 10:09:14,737|openeo_gfmap.manager|DEBUG:  Number of polygons to extract: 50


DataCube(<PGNode 'dimension_labels' at 0x7fcbbc7db950>)


2024-03-01 10:09:31,864|openeo_gfmap.manager|DEBUG:  Number of polygons to extract: 60


...

2024-03-01 10:09:52,061|openeo_gfmap.manager|DEBUG:  Number of polygons to extract: 20


DataCube(<PGNode 'dimension_labels' at 0x7fcbbc7d58b0>)


2024-03-01 10:10:12,419|openeo_gfmap.manager|DEBUG:  Number of polygons to extract: 38


DataCube(<PGNode 'dimension_labels' at 0x7fcbbc7d5c70>)


2024-03-01 10:11:45,651|openeo_gfmap.manager|DEBUG:  Updating status. 4 on 4 active jobs... -> DF columns: Index(['backend_name', 'out_prefix', 'out_extension', 'start_date', 'end_date',
       'geometry', 'nb_polygons', 'status', 'id', 'start_time', 'cpu',
       'memory', 'duration', 'description', 'costs'],
      dtype='object')
2024-03-01 10:11:45,944|openeo_gfmap.manager|DEBUG:  Status of job j-24030178c3b64df896d1924bfeef1ffb is running (on backend cdse).
2024-03-01 10:11:46,650|openeo_gfmap.manager|DEBUG:  Status of job j-2403019c533442d2a39daa06409bf225 is running (on backend cdse).
2024-03-01 10:11:47,018|openeo_gfmap.manager|DEBUG:  Status of job j-24030154ae7f4f79a0e41755c15284d1 is running (on backend cdse).
2024-03-01 10:11:50,213|openeo_gfmap.manager|DEBUG:  Status of job j-2403016e5d90427ca54853a4756c7f4b is queued (on backend cdse).
2024-03-01 10:12:50,870|openeo_gfmap.manager|DEBUG:  Updating status. 4 on 4 active jobs... -> DF columns: Index(['backend_name', 'out_pref

In [12]:
# Loads a geoparquet file from an artifactory public url
import fsspec

with fsspec.open("https://artifactory.vgt.vito.be/artifactory/auxdata-public/gfmap-temp/openeogfmap_dataframe_1.parquet") as file:
    gdf = gpd.read_parquet(file)

gdf.head()

,geometry,sample_id,landcover_label,croptype_label,irrigation_label,confidence,extract,valid_date,ref_id,h3index
0,"POLYGON ((638120.000 5289160.000, 638120.000 5...",at2021lpis390713,11,4380,0,None,True,2021-06-01,2021_EUR_DEMO_POLY_110,841e025ffffffff
1,"POLYGON ((649000.000 5284840.000, 649000.000 5...",at2021lpis2031299,11,4380,0,None,True,2021-06-01,2021_EUR_DEMO_POLY_110,841e025ffffffff
2,"POLYGON ((651040.000 5285480.000, 651040.000 5...",at2021lpis1150853,11,4380,0,None,True,2021-06-01,2021_EUR_DEMO_POLY_110,841e025ffffffff


In [13]:
gdf.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
from tempfile import NamedTemporaryFile

connection = cdse_connection()
job = connection.job('j-24022945a4e5492dbb40a61fca0cc91f')
tempfile = NamedTemporaryFile()
for asset in job.get_results().get_assets():
    asset.download(tempfile.name)


Authenticated using refresh token.


In [15]:
import xarray as xr

inopt = xr.open_dataset(tempfile.name)
inopt

<xarray.Dataset>
Dimensions:              (t: 167, x: 64, y: 64)
Coordinates:
  * t                    (t) datetime64[ns] 2020-09-01 2020-09-06 ... 2022-03-02
  * x                    (x) float64 5.05e+05 5.051e+05 ... 5.057e+05 5.057e+05
  * y                    (y) float64 6.361e+06 6.361e+06 ... 6.361e+06 6.361e+06
Data variables: (12/15)
    crs                  |S1 ...
    S2-B01               (t, y, x) float32 ...
    S2-B02               (t, y, x) float32 ...
    S2-B03               (t, y, x) float32 ...
    S2-B04               (t, y, x) float32 ...
    S2-B05               (t, y, x) float32 ...
    ...                   ...
    S2-B8A               (t, y, x) float32 ...
    S2-B09               (t, y, x) float32 ...
    S2-B11               (t, y, x) float32 ...
    S2-B12               (t, y, x) float32 ...
    S2-SCL               (t, y, x) float32 ...
    S2-SCL_DILATED_MASK  (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.27.1a1
    description:  
    title: